<a href="https://colab.research.google.com/github/hsabaghpour/AI_AGENTS_REPO/blob/main/implementing_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use termcolor to make it easy to colorize the outputs.
!pip install termcolor > /dev/null
!pip install langchain
!pip install openai
!pip install langchain_experimental
!pip install tiktoken
!pip install faiss-cpu==1.7.4
from datetime import datetime, timedelta
from typing import List
import math
import faiss
import os
import logging
logging.basicConfig(level=logging.ERROR)
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from termcolor import colored
from langchain_experimental.generative_agents import (

    GenerativeAgent,
    GenerativeAgentMemory,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.6 MB/s eta 0:00:00


In [8]:
os.environ["OPENAI_API_KEY"] = ''

In [9]:
USER_NAME = "Amir"  # The name you want to use when interviewing the agent.

LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.

## Implementing Your First Generative Agent




In [10]:


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OpenAIEmbeddings()
    # Initialize the vectorstore as empty
    embedding_size = 1536
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [11]:
alexis_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

# Defining the Generative Agent: Alexis
alexis = GenerativeAgent(
    name="Alexis",
    age=30,
    traits="curious, creative writer, world traveler",  # Persistent traits of Alexis
    status="exploring the intersection of technology and storytelling",  # Current status of Alexis
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=alexis_memory,
)

In [12]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(alexis.get_summary())

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is a straightforward and honest individual who values integrity and authenticity.


In [15]:
# We can add memories directly to the memory object

alexis_observations = [
    "Alexis recalls her morning walk in the park",
    "Alexis feels excited about the new book she started reading",
    "Alexis remembers her conversation with a close friend",
    "Alexis thinks about the painting she saw at the art gallery",
    "Alexis is planning to learn a new recipe for dinner",
    "Alexis is looking forward to her weekend trip",
    "Alexis contemplates her goals for the month.",
    "Alexis also love to learn Persian Cuisine",
    "Alexis is Ski avid, and enjoy snowboarding too",
    "Alexis hates insects",
    "Alexis just broke up with her boyfriend and would like to date new people after a griefing period"
]

for observation in alexis_observations:
    alexis.memory.add_memory(observation)



# We will see how this summary updates after more observations to create a more rich description.
print(alexis.get_summary(force_refresh=True))

Name: Alexis (age: 30)
Innate traits: curious, creative writer, world traveler
Alexis is a reflective and goal-oriented individual who enjoys outdoor activities like skiing and snowboarding. She values her close relationships and is open to new experiences, such as dating after a breakup. She also enjoys taking walks in the park and looks forward to weekend trips.


## Interacting and Providing Context to Generative Characters

## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [16]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [17]:
interview_agent(alexis, "What do you like to do?")


'Alexis said "I enjoy outdoor activities like skiing and snowboarding, exploring new cuisines, reading, and taking walks in the park. How about you?"'

In [18]:
interview_agent(alexis, "I am single and I want to date someone.")


'Alexis said "It\'s great that you\'re open to dating! I think it\'s important to be open to new experiences and meeting new people. Have you been on any interesting dates recently?"'

In [19]:
interview_agent(alexis, "I am searching, are single or you are in a relationship?")

'Alexis said "I recently went through a breakup, so I\'m currently single and open to dating new people. How about you?"'

In [20]:
interview_agent(alexis, "me 2, hey I know a cool place to go, musem of insects are you in?")

'Alexis said "Thanks for the suggestion, but I actually have a strong aversion to insects. Maybe we can explore a different museum or activity together sometime. Have a great day!"'

In [21]:
interview_agent(alexis, "oh I see, how about winter activiy, I enjoy skiing?")

'Alexis said "That\'s great, I love skiing too! It\'s such a fun winter activity. Have you been to any ski resorts recently?"'

## Step through the day's observations.

In [24]:
# Let's give Alexis a series of observations to reflect on her day
# Adding observations to Alexis' memory
alexis_observations_day = [
    "Alexis starts her day with a refreshing yoga session.",
    "Alexis spends time writing in her journal.",
    "Alexis experiments with a new recipe she found online.",
    "Alexis gets lost in her thoughts while gardening.",
    "Alexis decides to call her grandmother for a heartfelt chat.",
    "Alexis relaxes in the evening by playing her favorite piano pieces.",
    "Alexis decided to learn wild lif photgraphy so she has to buy a tele lens 200-600 mm"
]

for observation in alexis_observations_day:
    alexis.memory.add_memory(observation)


In [25]:
# Let's observe how Alexis's day influences her memory and character
for i, observation in enumerate(alexis_observations_day):
    _, reaction = alexis.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % len(alexis_observations_day)) == 0:
        print("*" * 40)
        print(
            colored(
                f"After these observations, Alexis's summary is:\n{alexis.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)


Alexis starts her day with a refreshing yoga session. Alexis smiles and feels energized after her yoga session.
Alexis spends time writing in her journal. Alexis feels a sense of calm and fulfillment as she writes in her journal.
Alexis experiments with a new recipe she found online. Alexis feels excited to try out the new recipe and looks forward to the cooking process.
Alexis gets lost in her thoughts while gardening. Alexis feels a sense of peace and tranquility while getting lost in her thoughts while gardening.
Alexis decides to call her grandmother for a heartfelt chat. Alexis smiles and feels grateful for the opportunity to connect with her grandmother.
Alexis relaxes in the evening by playing her favorite piano pieces. Alexis feels a sense of peace and contentment as she plays her favorite piano pieces.
Alexis decided to learn wild lif photgraphy so she has to buy a tele lens 200-600 mm Alexis feels excited about learning wildlife photography and looks forward to experimenting 